# Redes Neurais Recorrentes

Long-Short Term Memory (LSTM) são um tipo de rede neural, mais especificamente um tipo de rede neural recorrente.

Redes neurais recorrentes possuem uma característica distinta de redes neurais feedforward. Ao invés da informação seguir um fluxo contínuo sempre em um direção (usualmente para "frente"), as RNNs passam a informação também de volta ("trás"). Isso permite que essas simulem uma memória, sendo capazes de lidar melhor com problemas que variam com o tempo, como é o nosso caso.

In [1]:
import torch
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from context import fakenews
from fakenews import preprocess as pre
import gensim
import math

In [2]:
base = '/home/thales/dev/fakenews/'
news, labels = pre.run(base + 'data/Fake.csv', base + 'data/True.csv')
pre.truncate_news(news)

Succesfully read data from:
Fakes: /home/thales/dev/fakenews/data/Fake.csv
Reals: /home/thales/dev/fakenews/data/True.csv
Removing rows without text...
Removing publisher information...
Adding class column...
Merging fakes and reals
Merging titles and bodies...
Removing subjects and date...
Tokenizing data...
Truncating at 869


In [3]:
small_data = ['trump may leave the white house next year',
              'corona virus causes respiratory issues',
              'hospitals have no patients infected with new virus']

In [4]:
w2v = gensim.models.Word2Vec.load('fakenews-w2v.model')

In [5]:
def seq2vec(seqs):
    data = []
    for seq in seqs:
        vec_seq = []
        for word in seq.split(' '):
            if word in w2v.wv:
                vec_seq.append(w2v.wv[word])
        data.append(np.array(vec_seq).copy())
    return data

In [6]:
seq2vec(small_data)[0][:, :4]

array([[ 1.8907309e-01, -6.1274016e-01,  1.1100110e+00, -2.4788096e+00],
       [ 1.6640684e-02,  6.2638223e-02, -1.6004701e+00,  8.4372061e-01],
       [ 9.7744155e-04,  1.3789764e-01, -5.0027990e-01,  1.1279483e+00],
       [ 1.9337800e+00, -4.2167101e+00,  2.6433957e+00,  2.3664207e+00],
       [ 2.7858514e-01,  2.8959017e+00,  2.6175920e-02,  4.3230334e-01],
       [ 7.6738346e-01, -2.8276157e+00,  7.1034896e-01,  1.7026479e+00],
       [-2.0402887e+00, -4.7774467e-01, -2.8270140e+00,  1.5130302e+00]],
      dtype=float32)

In [7]:
vec_small_data = seq2vec(small_data)
small_tensor_data = [torch.from_numpy(vec) for vec in vec_small_data]
print(small_tensor_data[0][:, :4])

tensor([[ 1.8907e-01, -6.1274e-01,  1.1100e+00, -2.4788e+00],
        [ 1.6641e-02,  6.2638e-02, -1.6005e+00,  8.4372e-01],
        [ 9.7744e-04,  1.3790e-01, -5.0028e-01,  1.1279e+00],
        [ 1.9338e+00, -4.2167e+00,  2.6434e+00,  2.3664e+00],
        [ 2.7859e-01,  2.8959e+00,  2.6176e-02,  4.3230e-01],
        [ 7.6738e-01, -2.8276e+00,  7.1035e-01,  1.7026e+00],
        [-2.0403e+00, -4.7774e-01, -2.8270e+00,  1.5130e+00]])


In [8]:
class ElmanRNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din + dh, dout)

In [9]:
class ElmanRNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din + dh, dh)
        self.hidden = torch.nn.Linear(dh, dout)
        self.func = torch.sigmoid

In [53]:
class ElmanRNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.embeeding = w2v
        self.input = torch.nn.Linear(din + dh, dh)
        self.hidden = torch.nn.Linear(dh, dout)
        self.func = torch.sigmoid
        
    def forward(self, X, hidden):
        xt_ht = torch.cat((X, hidden), 0)
        hidden = self.func(self.input(xt_ht))
        output = self.hidden(hidden)
        return self.func(output), hidden

In [54]:
criterion = torch.nn.functional.mse_loss

In [57]:
def train(model, X, y, optim, nepochs):
    for epoch in range(nepochs):
        optim.zero_grad()
        outputs = []
        for new, lbl in zip(X, y):
            hidden = torch.zeros(50)
            for word in new:
                output, hidden = model(word, hidden)
            loss = criterion(output, lbl)
            loss.backward()
            print(f"{epoch} Loss = {loss}")
            optim.step()

In [58]:
elman_rnn = ElmanRNN(100, 50, 1)
sgd = torch.optim.SGD(elman_rnn.parameters(), lr=5e-3)
y = torch.tensor([0., 1., 0.]).view(3, -1)
train(elman_rnn, small_tensor_data, y, sgd, 10)

0 Loss = 0.1570308655500412
0 Loss = 0.32378053665161133
0 Loss = 0.1612144410610199
1 Loss = 0.1484067291021347
1 Loss = 0.3238369822502136
1 Loss = 0.15681371092796326
2 Loss = 0.14053837954998016
2 Loss = 0.3234764635562897
2 Loss = 0.15282148122787476
3 Loss = 0.13335317373275757
3 Loss = 0.32272768020629883
3 Loss = 0.14919863641262054
4 Loss = 0.12678398191928864
4 Loss = 0.32161903381347656
4 Loss = 0.14590942859649658
5 Loss = 0.12076929956674576
5 Loss = 0.32017868757247925
5 Loss = 0.14292128384113312
6 Loss = 0.11525321751832962
6 Loss = 0.31843388080596924
6 Loss = 0.14020460844039917
7 Loss = 0.11018522828817368
7 Loss = 0.3164110779762268
7 Loss = 0.13773268461227417
8 Loss = 0.10552000254392624
8 Loss = 0.31413528323173523
8 Loss = 0.1354813277721405
9 Loss = 0.10121671855449677
9 Loss = 0.3116307556629181
9 Loss = 0.13342860341072083


In [ ]:
# treina 1 em 1

Enquanto que para usar todos os elementos precisamos concatenar nossos dados.

In [ ]:
# todos de uma vez

Principalmente em NLP, a transformação do texto para representação numérica é bastante comum. Dessa forma, podemos criar uma LSTM customizada ao adicionar um layer de embedding que fica responsável por essa transformação.

In [ ]:
class LSTMfnews:
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Assim como podemos usar o novo layer durante o treinamento, aprendendo os embeddings de acordo com a task.

In [ ]:
# treina embedding + lstm

Claro que também podemos criar vários franksteins, da mesma forma que fizemos com a RNN. Note que o *gradient descent* é totalmente genérico. Isto é, ele não depende da arquitetura do seu modelo, apenas que as funções de ativação sejam deriváveis ou deriváveis por partes. Por isso podemos criar estruturas totalmente **idiotas**, com valores indo, voltando, pulando, ignorando, com muito layer, e com o que quisermos.

In [ ]:
# cria frankstein

É importante notar que redes profundas tendem a ocasionar *overfitting*, assim como os problemas decorrentes do algoritmo de aprendizado:

* Vanishing
* Exploding